# Pattern Visualization Helper

This notebook loads detections from the `detection_v1` exports, extracts the window
metadata for a chosen pattern/patient/date, fetches the raw CGM readings, and plots
the segment so you can visually inspect why the pattern fired.


In [1]:
# --- Parameters: edit these values ---
pattern_id = "frequent_spike"
patient_id = "6663751ba288866831d13caf"
detection_file = "detection_v1/0_detections.json"  # path to JSON export
service_date = "2025-07-18"  # date to visualise
example_index = 0            # index within the evidence examples list
minutes_padding = 60         # add +/- this many minutes around the example window
start_override = None        # optional ISO timestamp override for plot start
end_override = None          # optional ISO timestamp override for plot end


In [2]:
import os
import sys
from datetime import datetime, timezone, timedelta
from typing import Any, Dict, List, Optional

import pandas as pd
import plotly.graph_objects as go

# Ensure repository root is on sys.path
repo_root = os.path.abspath("..")
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from cgm_patterns.CGM_fetcher import iter_cgm_days
from cgm_patterns.models import CGMDay

TIMESTAMP_KEYS = [
    "start_time",
    "end_time",
    "recovery_time",
    "baseline_time",
    "peak_time",
]

WINDOW_HOUR_KEYS = [
    "window_start_hour",
    "window_end_hour",
]


def _parse_iso(ts: str) -> datetime:
    dt = datetime.fromisoformat(ts.replace("Z", "+00:00"))
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return dt


def _load_detections(path: str) -> List[Dict[str, Any]]:
    raw = json.loads(Path(path).read_text())
    detections = []
    for patient_payload in raw.values():
        for detections_list in patient_payload.get("detections", {}).values():
            detections.extend(detections_list)
    return detections


def _find_detection_examples(
    detections: List[Dict[str, Any]],
    pattern_id: str,
    service_date: str,
) -> List[Dict[str, Any]]:
    matches: List[Dict[str, Any]] = []
    for payload in detections:
        if payload.get("pattern_id") != pattern_id:
            continue
        evidence = payload.get("evidence", {})
        for example in evidence.get("examples", []):
            if example.get("service_date") == service_date:
                matches.append(example)
    return matches


def _extract_time_bounds(example: Dict[str, Any]) -> tuple[Optional[datetime], Optional[datetime]]:
    timestamps: List[datetime] = []
    for key in TIMESTAMP_KEYS:
        value = example.get(key)
        if isinstance(value, str):
            try:
                timestamps.append(_parse_iso(value))
            except ValueError:
                pass
    if not timestamps:
        return None, None
    return min(timestamps), max(timestamps)


def _fetch_cgm_frame(
    patient_id: str,
    start: Optional[datetime],
    end: Optional[datetime],
) -> pd.DataFrame:
    days: List[CGMDay] = list(iter_cgm_days(patient_id, start=start, end=end))
    frames = []
    for day in days:
        df = day.readings.copy()
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
        df["service_date"] = day.service_date
        frames.append(df)
    if not frames:
        return pd.DataFrame(columns=["timestamp", "glucose_mg_dL", "service_date"])
    combined = pd.concat(frames, ignore_index=True)
    combined = combined.sort_values("timestamp")
    if start:
        combined = combined[combined["timestamp"] >= start]
    if end:
        combined = combined[combined["timestamp"] <= end]
    return combined


def plot_cgm_segment(frame: pd.DataFrame, example: Dict[str, Any]) -> None:
    if frame.empty:
        print("No CGM readings found for the requested window.")
        return
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=frame["timestamp"],
            y=frame["glucose_mg_dL"],
            mode="lines+markers",
            name="CGM",
        )
    )

    for key in TIMESTAMP_KEYS:
        value = example.get(key)
        if isinstance(value, str):
            try:
                ts = _parse_iso(value)
                fig.add_vline(
                    x=ts,
                    line_dash="dash",
                    line_color="red",
                    annotation_text=key,
                    annotation_position="top right",
                )
            except ValueError:
                continue
    fig.update_layout(
        title=f"{example.get('service_date')} | {pattern_id} example",
        xaxis_title="Time",
        yaxis_title="Glucose (mg/dL)",
        legend=dict(orientation="h"),
    )
    fig.show()


ModuleNotFoundError: No module named 'plotly'

In [ ]:
detections = _load_detections(detection_file)
examples = _find_detection_examples(detections, pattern_id, service_date)
if not examples:
    raise ValueError("No evidence examples found for the given pattern/service date.")
example = examples[example_index]
print("Selected example keys:", list(example.keys()))
print(json.dumps(example, indent=2))


In [ ]:
start_ts, end_ts = _extract_time_bounds(example)
if start_override:
    start_ts = _parse_iso(start_override)
if end_override:
    end_ts = _parse_iso(end_override)

if start_ts and end_ts:
    padded_start = start_ts - timedelta(minutes=minutes_padding)
    padded_end = end_ts + timedelta(minutes=minutes_padding)
elif start_ts:
    padded_start = start_ts - timedelta(minutes=minutes_padding)
    padded_end = start_ts + timedelta(minutes=minutes_padding)
else:
    padded_start = _parse_iso(f"{service_date}T00:00:00+00:00")
    padded_end = _parse_iso(f"{service_date}T23:59:59+00:00")

cgm_frame = _fetch_cgm_frame(patient_id, padded_start, padded_end)
print(f"Fetched {len(cgm_frame)} CGM points between {padded_start} and {padded_end}.")
plot_cgm_segment(cgm_frame, example)
